In [42]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys
import json

In [43]:
zz=openai.Model.list()
len(zz['data'])

62

In [44]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

# Load template and descriptor file

In [45]:
sys.path.append('..')
from utils import *

In [106]:
prompt_dict=load_prompt_template('../prompt_template.md')

In [194]:
with open('2111.01152.jsonl','r') as f:
    kwargs= [json.loads(line) for line in f]


In [108]:
prompts=[generate_prompt(kwarg,prompt_dict=prompt_dict) for kwarg in kwargs]

In [235]:
import re
import string
def format_check(s, kwargs):
    placeholders=  [v[1] for v in (string.Formatter().parse(s)) if v[1] is not None]
    extra_args = set(kwargs.keys()) - set(placeholders+ ['task','answer'])  # Check for extra args.
    return extra_args


In [236]:
[format_check(prompt_dict[kwarg['task']],kwarg) for kwarg in kwargs]

[set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set()]

# LLM

In [67]:
def extract_var(prompt):
    string='Use the following conventions for the symbols'
    contains=False
    var_list=[]
    for line in prompt.split('\n'):
        if contains:
            if len(line.strip())==0:
                break
            var_list.append(line)

        if line.startswith(string):
            contains=True
        
    return var_list

In [120]:
def summarizer(summarization, prompt, response):
    '''Summarize the background (summarization) + question (prompt) + answer (response)'''
    var_old=extract_var(summarization)
    var_new=extract_var(prompt)
    var=var_old+var_new
    
    summarization_prompt=prompt_dict['Conversation summarizer'].format(background=summarization,question=prompt, answer=response)
    messages= [{'role':'user','content': summarization_prompt}]
    rs= openai.ChatCompletion.create(messages=messages, **model_params)

    summarized=rs['choices'][0]['message'].content

    
    if len(var)>0:
        if 'Use the following conventions for the symbols:  ' in summarized:
            summarized += '\n'+'\n'.join(var)
        else:
            summarized += '\n\nUse the following conventions for the symbols:  \n' +'\n'.join(var)
    return summarized

In [133]:
summarized=summarizer(summarization=summarization,
prompt=prompt,
response=response)

In [134]:
print(summarized)

The question asks to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space in the single-particle form. The system's degrees of freedom are the valley index (+K and -K valley) and the layer index (top and bottom layer). The kinetic Hamiltonian $H_{Kinetic}(r)$ is to be expressed using $E_{\tau,l}$, which are only on the diagonal terms, and the basis is arranged in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). The answer provides the kinetic Hamiltonian as a 4x4 diagonal matrix, where the diagonal elements represent the energy dispersion for each combination of layer and valley index. The off-diagonal elements are zero, indicating that transitions between different layers or valleys are not part of the kinetic energy term.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{l,\tau}$ : e

In [156]:
def solver(summarization, prompt):
    '''
    Solve the problem in the prompt
    '''
    sys_msg=[{'role': 'system', 'content': prompt_dict['Problem-solver']}]
    question_prompt='**Background**  \n{background}\n\n**Question**  \n{question}'.format(background=summarization,question=prompt)
    user_msg=[{'role':'user','content':question_prompt}]
    messages = sys_msg + user_msg
    rs= openai.ChatCompletion.create(messages=messages, **model_params)

    response=rs['choices'][0]['message'].content
    return response

In [149]:
response=solver(summarization=summarization,prompt=prompt)

In [150]:
Markdown(response[1]['content'])

**Background**  


**Question**  
You will be instructed to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   
The degrees of freedom of the system are: valley index (+K and -K valley), layer index (top and bottom layer). 
Express the Kinetic Hamiltonian $H_{Kinetic}(r)$ using $E_{\tau,l}$ which are only on the diagonal terms, and arrange the basis in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top).

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{l,\tau}$ : energy dispersion for layer $l$ and valley $\tau$  

In [141]:
print(response)

The kinetic Hamiltonian in real space for the hole-doped AB-stacked MoTe2/WSe2 system can be written in the single-particle form as a 4x4 matrix, considering the degrees of freedom of the system. The basis is arranged in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). 

The Hamiltonian $H_{Kinetic}(r)$ is then:

$H_{Kinetic}(r) = \begin{pmatrix} E_{b,+K} & 0 & 0 & 0 \\ 0 & E_{t,+K} & 0 & 0 \\ 0 & 0 & E_{b,-K} & 0 \\ 0 & 0 & 0 & E_{t,-K} \end{pmatrix}$

Here, $E_{l,\tau}$ represents the energy dispersion for layer $l$ and valley $\tau$. The diagonal terms of the Hamiltonian matrix represent the kinetic energy in each state. The off-diagonal terms are zero, indicating no coupling between different states in the kinetic term.


## Construct Kinetic Hamiltonian (continuum version, single-particle)

In [157]:
summarization=''
prompt=prompts[0]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

## Define each term in Kinetic Hamiltonian (continuum version)


In [159]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

In [161]:
prompt=prompts[1]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

## Construct Potential Hamiltonian (continuum version)

In [163]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem involves the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The system's degrees of freedom are the valley index (+K and -K valley) and layer index (top and bottom layer). The Kinetic Hamiltonian $H_{Kinetic}(r)$ is expressed as a 4x4 matrix with diagonal terms $E_{\tau,l}$, representing the energy dispersion for layer $l$ and valley $\tau$. The off-diagonal elements are zero, indicating no coupling between different states in the kinetic term.

The energy dispersion for free holes in each layer and valley is given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. For the top layer, there is a shift of momentum $k$ to make the band bottom at $\tilde{\tau}\kappa$, where $\tilde{\tau}$ is +1 for $\tau=+K$ and -1 for $\tau=-K$. Therefore, the energy dispersion for the top layer is $E_{\tau,t}(k)= -\frac{\hbar^2 (k-\tilde{\tau}\kappa)^2}{2 m_t}$.

Substituting these into the kinetic Hamiltonian, we get:

$H_{Kinetic}(r) = \begin{pmatrix} -\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\ 0 & -\frac{\hbar^2 (k-\tilde{\tau}\kappa)^2}{2 m_t} & 0 & 0 \\ 0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\ 0 & 0 & 0 & -\frac{\hbar^2 (k+\tilde{\tau}\kappa)^2}{2 m_t} \end{pmatrix}$

where $k=-i \partial_r$ is the momentum operator in real space.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone

In [164]:
prompt=prompts[2]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

The potential Hamiltonian $H_{Potential}(r)$ can be expressed as a 4x4 matrix in real space, similar to the kinetic Hamiltonian. Each 2x2 block corresponds to a valley, with the diagonal elements representing the intralayer potential and the off-diagonal elements representing the interlayer potential. The potential Hamiltonian is given by:

$H_{Potential}(r) = \begin{pmatrix} \Delta_b(r) & \Delta_{T,+K}(r) & 0 & 0 \\ \Delta_{T,+K}^*(r) & \Delta_t(r) & 0 & 0 \\ 0 & 0 & \Delta_b(r) & \Delta_{T,-K}(r) \\ 0 & 0 & \Delta_{T,-K}^*(r) & \Delta_t(r) \end{pmatrix}$

where $\Delta_b(r)$ and $\Delta_t(r)$ are the intralayer potentials for the bottom and top layers respectively, $\Delta_{T,+K}(r)$ and $\Delta_{T,-K}(r)$ are the interlayer potentials for the +K and -K valleys respectively, and the asterisk denotes the complex conjugate, ensuring the hermiticity of the Hamiltonian. The zero elements represent the absence of hybridization between the +K and -K valleys.

## Define each term in Potential Hamiltonian (continuum version)

In [165]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is expressed as a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley. The energy dispersion for free holes in each layer and valley is given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. For the top layer, there is a shift of momentum $k$ to make the band bottom at $\tilde{\tau}\kappa$. The kinetic Hamiltonian is then given by:

$H_{Kinetic}(r) = \begin{pmatrix} -\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\ 0 & -\frac{\hbar^2 (k-\tilde{\tau}\kappa)^2}{2 m_t} & 0 & 0 \\ 0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\ 0 & 0 & 0 & -\frac{\hbar^2 (k+\tilde{\tau}\kappa)^2}{2 m_t} \end{pmatrix}$

The potential Hamiltonian $H_{Potential}(r)$ is a 4x4 matrix in real space, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. The potential Hamiltonian is given by:

$H_{Potential}(r) = \begin{pmatrix} \Delta_b(r) & \Delta_{T,+K}(r) & 0 & 0 \\ \Delta_{T,+K}^*(r) & \Delta_t(r) & 0 & 0 \\ 0 & 0 & \Delta_b(r) & \Delta_{T,-K}(r) \\ 0 & 0 & \Delta_{T,-K}^*(r) & \Delta_t(r) \end{pmatrix}$

where $\Delta_b(r)$ and $\Delta_t(r)$ are the intralayer potentials for the bottom and top layers respectively, $\Delta_{T,+K}(r)$ and $\Delta_{T,-K}(r)$ are the interlayer potentials for the +K and -K valleys respectively. The zero elements represent the absence of hybridization between the +K and -K valleys.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.

In [166]:
prompt=prompts[3]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

The intralayer potential for the top layer $\Delta_t(r)$ is given by $V_{zt}$.

The intralayer potential for the bottom layer $\Delta_b(r)$ is given by $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.

The interlayer tunneling at the +K valley $\Delta_{T,+K}(r)$ is given by $w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r})$.

The interlayer tunneling at the -K valley $\Delta_{T,-K}(r)$ is given by $w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r})$.

Substituting these expressions into the potential Hamiltonian $H_{Potential}(r)$, we get:

$H_{Potential}(r) = \begin{pmatrix} 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & 0 & 0 \\ w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) & V_{zt} & 0 & 0 \\ 0 & 0 & 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) \\ 0 & 0 & w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & V_{zt} \end{pmatrix}$

## Convert from single-particle to second-quantized form, return in matrix


In [167]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The question asks to construct each term of the potential Hamiltonian, namely, the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$. The intralayer potential for the top layer $\Delta_t(r)$ is given by $V_{zt}$, and for the bottom layer $\Delta_b(r)$ is given by $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$. The interlayer tunneling at the +K valley $\Delta_{T,+K}(r)$ is given by $w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r})$, and at the -K valley $\Delta_{T,-K}(r)$ is given by $w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r})$. Substituting these expressions into the potential Hamiltonian $H_{Potential}(r)$ gives the final form of the potential Hamiltonian.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength

In [168]:
prompt=prompts[4]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

The noninteracting Hamiltonian in the real space $H^{0}(r)$ is given by the sum of the kinetic and potential Hamiltonians. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, and the potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential.

The electron annihilation operator at valley index $\tau$ and layer index $l$ is denoted by $\psi_{\tau,l}(r)$, and the spinor containing all annihilation operators $\psi_{\tau,l}(r)$ is denoted by $\vec{\psi}(r)$.

The second quantized form of the total noninteracting Hamiltonian $\hat{H}^{0}$ can be obtained by sandwiching the single-particle Hamiltonian $H^{0}(r)$ between the creation and annihilation operators, and summing over the real space position $r$.

Therefore, the second quantized form of the total noninteracting Hamiltonian is given by:

$\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$

where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space.

## Convert from single-particle to second-quantized form, return in summation (expand the matrix)


In [169]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The question asks to construct the second quantized form of the total noninteracting Hamiltonian in the real space. The noninteracting Hamiltonian in the real space $H^{0}(r)$ is the sum of Kinetic Hamiltonian $H_{Kinetic}(r)$ and Potential Hamiltonian $H_{Potential}(r)$. The second quantized form of the total noninteracting Hamiltonian $\hat{H}^{0}$ can be obtained by sandwiching the single-particle Hamiltonian $H^{0}(r)$ between the creation and annihilation operators, and summing over the real space position $r$. Therefore, the second quantized form of the total noninteracting Hamiltonian is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$ where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 

In [170]:
prompt=prompts[5]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

The second-quantized form of the Hamiltonian $\hat{H}^{0}$ can be expanded as follows:

$\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$

where $\vec{\psi}(r) = \begin{pmatrix} \psi_{+K,b}(r) \\ \psi_{+K,t}(r) \\ \psi_{-K,b}(r) \\ \psi_{-K,t}(r) \end{pmatrix}$ and $\vec{\psi}^\dagger(r) = \begin{pmatrix} \psi_{+K,b}^\dagger(r) & \psi_{+K,t}^\dagger(r) & \psi_{-K,b}^\dagger(r) & \psi_{-K,t}^\dagger(r) \end{pmatrix}$.

We can then expand $\hat{H}^{0}$ as:

$\hat{H}^{0} = \int dr \sum_{\tau_1,l_1,\tau_2,l_2} \psi_{\tau_1,l_1}^\dagger(r) H^{0}_{\tau_1,l_1,\tau_2,l_2}(r) \psi_{\tau_2,l_2}(r)$

where $\tau_1, \tau_2 \in \{+K, -K\}$ and $l_1, l_2 \in \{t, b\}$.

Given the form of $H^{0}(r)$, we can see that the off-diagonal elements of the kinetic term are zero, and the off-diagonal elements of the potential term are zero when $\tau_1 \neq \tau_2$ or $l_1 = l_2$. Therefore, we can simplify the above expression to:

$\hat{H}^{0} = \int dr \sum_{\tau,l} \psi_{\tau,l}^\dagger(r) H^{0}_{\tau,l,\tau,l}(r) \psi_{\tau,l}(r) + \int dr \sum_{\tau} \left( \psi_{\tau,b}^\dagger(r) H^{0}_{\tau,b,\tau,t}(r) \psi_{\tau,t}(r) + \psi_{\tau,t}^\dagger(r) H^{0}_{\tau,t,\tau,b}(r) \psi_{\tau,b}(r) \right)$

where $\tau \in \{+K, -K\}$ and $l \in \{t, b\}$.

## Convert noninteracting Hamiltonian in real space to momentum space (continuum version)


In [171]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The question asks to construct the second quantized form of the total noninteracting Hamiltonian in the real space. The noninteracting Hamiltonian in the real space $H^{0}(r)$ is the sum of Kinetic Hamiltonian $H_{Kinetic}(r)$ and Potential Hamiltonian $H_{Potential}(r)$. The second quantized form of the total noninteracting Hamiltonian $\hat{H}^{0}$ can be obtained by sandwiching the single-particle Hamiltonian $H^{0}(r)$ between the creation and annihilation operators, and summing over the real space position $r$. Therefore, the second quantized form of the total noninteracting Hamiltonian is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$ where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space.

The second-quantized form of the Hamiltonian $\hat{H}^{0}$ can be expanded as follows:

$\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$

where $\vec{\psi}(r) = \begin{pmatrix} \psi_{+K,b}(r) \\ \psi_{+K,t}(r) \\ \psi_{-K,b}(r) \\ \psi_{-K,t}(r) \end{pmatrix}$ and $\vec{\psi}^\dagger(r) = \begin{pmatrix} \psi_{+K,b}^\dagger(r) & \psi_{+K,t}^\dagger(r) & \psi_{-K,b}^\dagger(r) & \psi_{-K,t}^\dagger(r) \end{pmatrix}$.

We can then expand $\hat{H}^{0}$ as:

$\hat{H}^{0} = \int dr \sum_{\tau_1,l_1,\tau_2,l_2} \psi_{\tau_1,l_1}^\dagger(r) H^{0}_{\tau_1,l_1,\tau_2,l_2}(r) \psi_{\tau_2,l_2}(r)$

where $\tau_1, \tau_2 \in \{+K, -K\}$ and $l_1, l_2 \in \{t, b\}$.

Given the form of $H^{0}(r)$, we can see that the off-diagonal elements of the kinetic term are zero, and the off-diagonal elements of the potential term are zero when $\tau_1 \neq \tau_2$ or $l_1 = l_2$. Therefore, we can simplify the above expression to:

$\hat{H}^{0} = \int dr \sum_{\tau,l} \psi_{\tau,l}^\dagger(r) H^{0}_{\tau,l,\tau,l}(r) \psi_{\tau,l}(r) + \int dr \sum_{\tau} \left( \psi_{\tau,b}^\dagger(r) H^{0}_{\tau,b,\tau,t}(r) \psi_{\tau,t}(r) + \psi_{\tau,t}^\dagger(r) H^{0}_{\tau,t,\tau,b}(r) \psi_{\tau,b}(r) \right)$

where $\tau \in \{+K, -K\}$ and $l \in \{t, b\}$.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  

In [172]:
prompt=prompts[6]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

Following the example, we can apply the Fourier transformation to the total noninteracting Hamiltonian $\hat{H}^{0}$ in the second quantized form. 

First, we define the Fourier transformation of $\psi_{\tau,l}^\dagger(r)$ and $\psi_{\tau,l}(r)$ as:

$c_{\tau,l}^\dagger(k)=\frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$

$c_{\tau,l}(k)=\frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}(r) e^{-i k \cdot r}$

This leads to the inverse Fourier transformation:

$\psi_{\tau,l}^\dagger(r) = \frac{1}{\sqrt{V}} \sum_{k_1} c_{\tau,l}^\dagger(k_1) e^{-i k_1 \cdot r}$

$\psi_{\tau,l}(r) = \frac{1}{\sqrt{V}} \sum_{k_2} c_{\tau,l}(k_2) e^{i k_2 \cdot r}$

Substituting these into $\hat{H}^{0}$, we get:

$\hat{H}^{0} = \int dr \sum_{\tau,l,k_1,k_2} c_{\tau,l}^\dagger(k_1) c_{\tau,l}(k_2) \frac{1}{V} e^{-i (k_1-k_2)\cdot r} H^{0}_{\tau,l,\tau,l}(r) + \int dr \sum_{\tau,k_1,k_2} \left( c_{\tau,b}^\dagger(k_1) c_{\tau,t}(k_2) \frac{1}{V} e^{-i (k_1-k_2)\cdot r} H^{0}_{\tau,b,\tau,t}(r) + c_{\tau,t}^\dagger(k_1) c_{\tau,b}(k_2) \frac{1}{V} e^{-i (k_1-k_2)\cdot r} H^{0}_{\tau,t,\tau,b}(r) \right)$

We can then define the Fourier transformation of $H^{0}_{\tau,l,\tau,l}(r)$, $H^{0}_{\tau,b,\tau,t}(r)$, and $H^{0}_{\tau,t,\tau,b}(r)$ as:

$H^{0}_{\tau,l,\tau,l}(k_1,k_2)=\frac{1}{V} \int dr e^{-i (k_1-k_2)\cdot r} H^{0}_{\tau,l,\tau,l}(r)$

$H^{0}_{\tau,b,\tau,t}(k_1,k_2)=\frac{1}{V} \int dr e^{-i (k_1-k_2)\cdot r} H^{0}_{\tau,b,\tau,t}(r)$

$H^{0}_{\tau,t,\tau,b}(k_1,k_2)=\frac{1}{V} \int dr e^{-i (k_1-k_2)\cdot r} H^{0}_{\tau,t,\tau,b}(r)$

Finally, we can write $\hat{H}^{0}$ in terms of $c_{\tau,l}^\dagger(k)$ and $c_{\tau,l}(k)$ as:

$\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} c_{\tau,l}^\dagger(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) c_{\tau,l}(k_2) + \sum_{\tau,k_1,k_2} \left( c_{\tau,b}^\dagger(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) c_{\tau,t}(k_2) + c_{\tau,t}^\dagger(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) c_{\tau,b}(k_2) \right)$

where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

## Particle-hole transformation


In [173]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The question asks to construct the second quantized form of the total noninteracting Hamiltonian in the real space. The noninteracting Hamiltonian in the real space $H^{0}(r)$ is the sum of Kinetic Hamiltonian $H_{Kinetic}(r)$ and Potential Hamiltonian $H_{Potential}(r)$. The second quantized form of the total noninteracting Hamiltonian $\hat{H}^{0}$ can be obtained by sandwiching the single-particle Hamiltonian $H^{0}(r)$ between the creation and annihilation operators, and summing over the real space position $r$. Therefore, the second quantized form of the total noninteracting Hamiltonian is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$ where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space.

The task is to convert the total noninteracting Hamiltonian in the second quantized form from the basis in real space to the basis by momentum space. To do this, the Fourier transformation is applied to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. The total noninteracting Hamiltonian $\hat{H}^{0}$ is then expressed in terms of $c_{\tau,l}^\dagger(k)$. 

The Fourier transformation of $H^{0}_{\tau,l,\tau,l}(r)$, $H^{0}_{\tau,b,\tau,t}(r)$, and $H^{0}_{\tau,t,\tau,b}(r)$ is defined and the total noninteracting Hamiltonian $\hat{H}^{0}$ is written in terms of $c_{\tau,l}^\dagger(k)$ and $c_{\tau,l}(k)$ as:

$\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} c_{\tau,l}^\dagger(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) c_{\tau,l}(k_2) + \sum_{\tau,k_1,k_2} \left( c_{\tau,b}^\dagger(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) c_{\tau,t}(k_2) + c_{\tau,t}^\dagger(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) c_{\tau,b}(k_2) \right)$

where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  
$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  

In [174]:
prompt=prompts[7]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

Following the example, the Hamiltonian $\hat{H}^{0}$ after the particle-hole transformation is:

$\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} b_{\tau,l}(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) b_{\tau,l}^\dagger(k_2) + \sum_{\tau,k_1,k_2} \left( b_{\tau,b}(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) b_{\tau,t}^\dagger(k_2) + b_{\tau,t}(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) b_{\tau,b}^\dagger(k_2) \right)$

where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

## Simplify the Hamiltonian in the particle-hole basis


In [175]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The task is to construct the second quantized form of the total noninteracting Hamiltonian in the real space and then convert it to the basis in momentum space using Fourier transformation. The total noninteracting Hamiltonian in the second quantized form is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space. The Fourier transformation is applied to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. The total noninteracting Hamiltonian $\hat{H}^{0}$ is then expressed in terms of $c_{\tau,l}^\dagger(k)$.

The question asks to perform a particle-hole transformation by defining a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$. The transformed Hamiltonian is $\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} b_{\tau,l}(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) b_{\tau,l}^\dagger(k_2) + \sum_{\tau,k_1,k_2} \left( b_{\tau,b}(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) b_{\tau,t}^\dagger(k_2) + b_{\tau,t}(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) b_{\tau,b}^\dagger(k_2) \right)$, where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  
$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$

In [176]:
prompt=prompts[8]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

Following the example, we can simplify the Hamiltonian $\hat{H}^{0}$ in the hole basis. 

First, we use the anticommuting relation for fermions, $[b_{\tau, l_1}(k_1), b_{\tau, l_2}^\dagger(k_2)]_{+} = \delta_{l_1,l_2}\delta_{k_1,k_2}$, which gives $b_{\tau, l_1}(k_1) b_{\tau, l_2}^\dagger(k_2) = \delta_{l_1,l_2}\delta_{k_1,k_2} - b_{\tau, l_2}^\dagger(k_2) b_{\tau, l_1}(k_1)$.

Substituting this into $\hat{H}^{0}$, we get:

$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} (\delta_{l_1,l_2}\delta_{k_1,k_2} - b_{\tau, l_2}^\dagger(k_2) b_{\tau, l_1}(k_1)) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2)$.

This simplifies to:

$\hat{H}^{0} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_2}^\dagger(k_2) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2) b_{\tau, l_1}(k_1)$.

Next, we relabel the indices $l_1$ and $l_2$ as well as $k_1$ and $k_2$ in the second term to make it consistent with the original order of indices. This gives:

$\hat{H}^{0} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) H^{0}_{\tau, l_2, \tau, l_1}(k_2, k_1) b_{\tau, l_2}(k_2)$.

Finally, we use the fact that $H^{0}_{\tau, l_2, \tau, l_1}(k_2, k_1) = (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^*$, because the Hamiltonian is Hermitian. This gives:

$\hat{H}^{0} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$.

Therefore, the final version of the noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification is:

$\hat{H}^{0,final} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$.

## Construct interaction Hamiltonian (momentum space)


In [177]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The task is to construct the second quantized form of the total noninteracting Hamiltonian in the real space and then convert it to the basis in momentum space using Fourier transformation. The total noninteracting Hamiltonian in the second quantized form is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space. The Fourier transformation is applied to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. The total noninteracting Hamiltonian $\hat{H}^{0}$ is then expressed in terms of $c_{\tau,l}^\dagger(k)$.

The question asks to perform a particle-hole transformation by defining a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$. The transformed Hamiltonian is $\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} b_{\tau,l}(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) b_{\tau,l}^\dagger(k_2) + \sum_{\tau,k_1,k_2} \left( b_{\tau,b}(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) b_{\tau,t}^\dagger(k_2) + b_{\tau,t}(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) b_{\tau,b}^\dagger(k_2) \right)$, where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

The task is to simplify the $\hat{H}^{0}$ in the hole basis using canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators. The final version of the noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification is:

$\hat{H}^{0,final} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  
$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$[b_i, b_j]_{+}$ : anticommuting relation, i.e., $[b_i, b_j]_+= b_i b_j +b_j b_i$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.

In [178]:
prompt=prompts[9]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

The interaction Hamiltonian $\hat{H}^{int}$ in the momentum space can be constructed as follows:

$\hat{H}^{int} = \frac{1}{2V} \sum_{\tau_1, \tau_2, l_1, l_2, k_1, k_2, k_3, k_4} b_{\tau_1, l_1}^\dagger(k_1) b_{\tau_2, l_2}^\dagger(k_2) V(k_1 - k_4) b_{\tau_2, l_2}(k_3) b_{\tau_1, l_1}(k_4) \delta_{k_1 + k_2, k_3 + k_4}$

where $\tau_1, \tau_2 \in \{+K, -K\}$, $l_1, l_2 \in \{t, b\}$, and $k_1, k_2, k_3, k_4$ are summed over the extended Brillouin zone. The Kronecker delta function $\delta_{k_1 + k_2, k_3 + k_4}$ ensures the conservation of total momentum. The interaction form is given by the dual-gate-screened Coulomb interaction $V(q)=2\pi e^2\tanh(|q| d)/(\epsilon |q|)$, where $q=k_1-k_4$ is the transferred total momentum. The normalization factor is $\frac{1}{2V}$, where $V$ is the area of the unit cell in the real space.

## Wick's theorem


In [179]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The task is to construct the second quantized form of the total noninteracting Hamiltonian in the real space and then convert it to the basis in momentum space using Fourier transformation. The total noninteracting Hamiltonian in the second quantized form is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space. The Fourier transformation is applied to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. The total noninteracting Hamiltonian $\hat{H}^{0}$ is then expressed in terms of $c_{\tau,l}^\dagger(k)$.

The question asks to perform a particle-hole transformation by defining a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$. The transformed Hamiltonian is $\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} b_{\tau,l}(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) b_{\tau,l}^\dagger(k_2) + \sum_{\tau,k_1,k_2} \left( b_{\tau,b}(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) b_{\tau,t}^\dagger(k_2) + b_{\tau,t}(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) b_{\tau,b}^\dagger(k_2) \right)$, where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

The task is to simplify the $\hat{H}^{0}$ in the hole basis using canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators. The final version of the noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification is:

$\hat{H}^{0,final} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$.

The interaction Hamiltonian $\hat{H}^{int}$ in the momentum space can be constructed as follows:

$\hat{H}^{int} = \frac{1}{2V} \sum_{\tau_1, \tau_2, l_1, l_2, k_1, k_2, k_3, k_4} b_{\tau_1, l_1}^\dagger(k_1) b_{\tau_2, l_2}^\dagger(k_2) V(k_1 - k_4) b_{\tau_2, l_2}(k_3) b_{\tau_1, l_1}(k_4) \delta_{k_1 + k_2, k_3 + k_4}$

where $\tau_1, \tau_2 \in \{+K, -K\}$, $l_1, l_2 \in \{t, b\}$, and $k_1, k_2, k_3, k_4$ are summed over the extended Brillouin zone. The Kronecker delta function $\delta_{k_1 + k_2, k_3 + k_4}$ ensures the conservation of total momentum. The interaction form is given by the dual-gate-screened Coulomb interaction $V(q)=2\pi e^2\tanh(|q| d)/(\epsilon |q|)$, where $q=k_1-k_4$ is the transferred total momentum. The normalization factor is $\frac{1}{2V}$, where $V$ is the area of the unit cell in the real space.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  
$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$[b_i, b_j]_{+}$ : anticommuting relation, i.e., $[b_i, b_j]_+= b_i b_j +b_j b_i$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.
$d$ : distance between dual gates  
$\epsilon$ : dielectric constant  
$V(q)$ : dual-gate-screened Coulomb interaction in the momentum space

In [180]:
prompt=prompts[10]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

Following the instructions and using Wick's theorem, the interaction term $\hat{H}^{int}$ can be expanded as follows:

$\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} b_{l_1,\tau_1}^\dagger(k_1)b_{l_2,\tau_2}^\dagger(k_2)b_{l_2,\tau_2}(k_3)b_{l_1,\tau_1}(k_4) V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

This can be expanded using Wick's theorem (as shown in EXAMPLE 1) to:

$\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[ \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) + \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

Preserving only the normal terms, we get:

$\hat{H}^{int,HF} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[ \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

This is the interaction term after Hartree-Fock approximation.

## Extract quadratic term


In [181]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The task is to construct the second quantized form of the total noninteracting Hamiltonian in the real space and then convert it to the basis in momentum space using Fourier transformation. The total noninteracting Hamiltonian in the second quantized form is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space. The Fourier transformation is applied to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. The total noninteracting Hamiltonian $\hat{H}^{0}$ is then expressed in terms of $c_{\tau,l}^\dagger(k)$.

The question asks to perform a particle-hole transformation by defining a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$. The transformed Hamiltonian is $\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} b_{\tau,l}(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) b_{\tau,l}^\dagger(k_2) + \sum_{\tau,k_1,k_2} \left( b_{\tau,b}(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) b_{\tau,t}^\dagger(k_2) + b_{\tau,t}(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) b_{\tau,b}^\dagger(k_2) \right)$, where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

The task is to simplify the $\hat{H}^{0}$ in the hole basis using canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators. The final version of the noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification is:

$\hat{H}^{0,final} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$.

The interaction Hamiltonian $\hat{H}^{int}$ in the momentum space can be constructed as follows:

$\hat{H}^{int} = \frac{1}{2V} \sum_{\tau_1, \tau_2, l_1, l_2, k_1, k_2, k_3, k_4} b_{\tau_1, l_1}^\dagger(k_1) b_{\tau_2, l_2}^\dagger(k_2) V(k_1 - k_4) b_{\tau_2, l_2}(k_3) b_{\tau_1, l_1}(k_4) \delta_{k_1 + k_2, k_3 + k_4}$

where $\tau_1, \tau_2 \in \{+K, -K\}$, $l_1, l_2 \in \{t, b\}$, and $k_1, k_2, k_3, k_4$ are summed over the extended Brillouin zone. The Kronecker delta function $\delta_{k_1 + k_2, k_3 + k_4}$ ensures the conservation of total momentum. The interaction form is given by the dual-gate-screened Coulomb interaction $V(q)=2\pi e^2\tanh(|q| d)/(\epsilon |q|)$, where $q=k_1-k_4$ is the transferred total momentum. The normalization factor is $\frac{1}{2V}$, where $V$ is the area of the unit cell in the real space.

The question asks to perform a Hartree-Fock approximation to expand the interaction term, $\hat{H}^{int}$. Using Wick's theorem, the interaction term $\hat{H}^{int}$ can be expanded as follows:

$\hat{H}^{int,HF} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[ \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

This is the interaction term after Hartree-Fock approximation.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  
$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$[b_i, b_j]_{+}$ : anticommuting relation, i.e., $[b_i, b_j]_+= b_i b_j +b_j b_i$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.
$d$ : distance between dual gates  
$\epsilon$ : dielectric constant  
$V(q)$ : dual-gate-screened Coulomb interaction in the momentum space
$\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2) \rangle$ : expectation of $b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2)$ over the many-body ground states.  
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation

In [182]:
prompt=prompts[11]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

The quadratic terms in $\hat{H}^{int,HF}$ are those that contain the product of a creation operator and an annihilation operator. From the given expression for $\hat{H}^{int,HF}$, the quadratic terms are:

$\hat{H}^{int,HF,2} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

The terms that are solely expectations or products of expectations are not included in $\hat{H}^{int,HF,2}$.

## Swap the index to combine Hartree and Fock terms


In [183]:
summarization=summarizer(summarization=summarization, prompt=prompt, response=response)
Markdown(summarization)

The problem discusses the kinetic and potential terms of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in real space. The kinetic Hamiltonian is a 4x4 matrix with diagonal terms representing the energy dispersion for each layer and valley, given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$. The potential Hamiltonian is also a 4x4 matrix, with each 2x2 block corresponding to a valley. The diagonal elements represent the intralayer potential and the off-diagonal elements represent the interlayer potential. 

The task is to construct the second quantized form of the total noninteracting Hamiltonian in the real space and then convert it to the basis in momentum space using Fourier transformation. The total noninteracting Hamiltonian in the second quantized form is given by: $\hat{H}^{0} = \sum_{r} \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $\vec{\psi}^\dagger(r)$ is the hermitian conjugate of $\vec{\psi}(r)$, and $H^{0}(r)$ is the single-particle noninteracting Hamiltonian in the real space. The Fourier transformation is applied to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. The total noninteracting Hamiltonian $\hat{H}^{0}$ is then expressed in terms of $c_{\tau,l}^\dagger(k)$.

The question asks to perform a particle-hole transformation by defining a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$. The transformed Hamiltonian is $\hat{H}^{0} = \sum_{\tau,l,k_1,k_2} b_{\tau,l}(k_1) H^{0}_{\tau,l,\tau,l}(k_1,k_2) b_{\tau,l}^\dagger(k_2) + \sum_{\tau,k_1,k_2} \left( b_{\tau,b}(k_1) H^{0}_{\tau,b,\tau,t}(k_1,k_2) b_{\tau,t}^\dagger(k_2) + b_{\tau,t}(k_1) H^{0}_{\tau,t,\tau,b}(k_1,k_2) b_{\tau,b}^\dagger(k_2) \right)$, where $\tau \in \{+K, -K\}$, $l \in \{t, b\}$, and $k_1, k_2$ are summed over the extended Brillouin zone.

The task is to simplify the $\hat{H}^{0}$ in the hole basis using canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators. The final version of the noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification is:

$\hat{H}^{0,final} = \sum_{\tau, l, k} H^{0}_{\tau, l, \tau, l}(k, k) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$.

The interaction Hamiltonian $\hat{H}^{int}$ in the momentum space can be constructed as follows:

$\hat{H}^{int} = \frac{1}{2V} \sum_{\tau_1, \tau_2, l_1, l_2, k_1, k_2, k_3, k_4} b_{\tau_1, l_1}^\dagger(k_1) b_{\tau_2, l_2}^\dagger(k_2) V(k_1 - k_4) b_{\tau_2, l_2}(k_3) b_{\tau_1, l_1}(k_4) \delta_{k_1 + k_2, k_3 + k_4}$

where $\tau_1, \tau_2 \in \{+K, -K\}$, $l_1, l_2 \in \{t, b\}$, and $k_1, k_2, k_3, k_4$ are summed over the extended Brillouin zone. The Kronecker delta function $\delta_{k_1 + k_2, k_3 + k_4}$ ensures the conservation of total momentum. The interaction form is given by the dual-gate-screened Coulomb interaction $V(q)=2\pi e^2\tanh(|q| d)/(\epsilon |q|)$, where $q=k_1-k_4$ is the transferred total momentum. The normalization factor is $\frac{1}{2V}$, where $V$ is the area of the unit cell in the real space.

The question asks to perform a Hartree-Fock approximation to expand the interaction term, $\hat{H}^{int}$. Using Wick's theorem, the interaction term $\hat{H}^{int}$ can be expanded as follows:

$\hat{H}^{int,HF} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[ \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

This is the interaction term after Hartree-Fock approximation.

The question asks to extract the quadratic terms in the $\hat{H}^{int,HF}$. The quadratic terms mean terms that are proportional to $b^\dagger b$, which excludes terms that are solely expectations or products of expectations. The quadratic terms in $\hat{H}^{int,HF}$ are:

$\hat{H}^{int,HF,2} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

The terms that are solely expectations or products of expectations are not included in $\hat{H}^{int,HF,2}$.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$
$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength
$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  
$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$[b_i, b_j]_{+}$ : anticommuting relation, i.e., $[b_i, b_j]_+= b_i b_j +b_j b_i$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.
$d$ : distance between dual gates  
$\epsilon$ : dielectric constant  
$V(q)$ : dual-gate-screened Coulomb interaction in the momentum space
$\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2) \rangle$ : expectation of $b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2)$ over the many-body ground states.  
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation
$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$

In [184]:
prompt=prompts[12]['content']
response=solver(summarization=summarization, prompt=prompt)
Markdown(response)

Following the example, we can simplify the quadratic term $\hat{H}^{int,HF,2}$ by relabeling the indices. 

For the second Hartree term, we swap the indices in the expected value and the indices in the quadratic operators, namely, $\tau_1 \leftrightarrow \tau_2$, $l_1 \leftrightarrow l_2$, $k_1 \leftrightarrow k_2$, and $k_3 \leftrightarrow k_4$. After the replacement, the second Hartree term becomes:

$\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3)$

Similarly, for the second Fock term, we swap the indices in the expected value and the indices in the quadratic operators, namely, $\tau_1 \leftrightarrow \tau_2$, $l_1 \leftrightarrow l_2$, $k_1 \leftrightarrow k_2$, and $k_3 \leftrightarrow k_4$. After the replacement, the second Fock term becomes:

$-\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_2,l_2}(k_3) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_1,l_1}(k_4)$

Therefore, the simplified $\hat{H}^{int,HF,2}$ is:

$\hat{H}^{int,HF,2} = \frac{1}{V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3) - \langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_2,l_2}(k_3) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_1,l_1}(k_4) \right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$

This simplification reduces the four terms (two Hartree and two Fock terms) to only two terms (one Hartree and one Fock term).

# In On-Go

In [185]:
answers=[]
for idx,prompt_i in enumerate(prompts):
    print(f'Asking {idx}..')
    prompt=prompt_i['content']
    if idx==0:
        summarization=''
        response=solver(summarization=summarization, prompt=prompt)
    else:
        summarization=summarizer(summarization=summarization, prompt=prompt, response=response)        
        response=solver(summarization=summarization, prompt=prompt)
    answers.append(response)
    

Asking 0..
Asking 1..
Asking 2..
Asking 3..
Asking 4..
Asking 5..
Asking 6..
Asking 7..
Asking 8..
Asking 9..
Asking 10..
Asking 11..
Asking 12..


In [190]:
string=''
for kwarg,prompt_i,answer in zip(kwargs,prompts,answers):
    task=kwarg['task']
    prompt=prompt_i['content']
    added=f'## {task}  \n**Prompt:**  \n{prompt}\n\n**Completion:**  \n{answer}\n\n'
    string+=added

with open('2111.01152_auto.md','w') as f:
    f.write(string)

In [17]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]

In [ ]:
user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [ ]:
# summarization=''
# summarization=r'''The discussion revolves around converting the noninteracting Hamiltonian in the second quantized form from real space to momentum space in a hole-doped AB-stacked MoTe2/WSe2 system. The noninteracting Hamiltonian in real space, $\hat{H}^{0}$, is represented by the sum of the kinetic and potential Hamiltonians and is expressed as $\hat{H}^{0} = \sum_r \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$. 

# To shift from real to momentum space, Fourier transformations of the creation and annihilation operators are applied. The Fourier transforms of these operators for this multilayer system are given by:

# $$c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$$
# $$c_{\tau,l}(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}(r) e^{-i k \cdot r}$$

# And their inverse Fourier transforms are:

# $$\psi_{\tau,l}^\dagger(r) = \frac{1}{\sqrt{V}} \sum_{k_1} c_{\tau,l}^\dagger(k_1) e^{-i k_1 \cdot r}$$
# $$\psi_{\tau,l}(r) = \frac{1}{\sqrt{V}} \sum_{k_2} c_{\tau,l}(k_2) e^{i k_2 \cdot r}$$

# Using these transforms, the noninteracting Hamiltonian in the second quantized form is converted to momentum space as:

# $$\hat{H}^{0} = \sum_{k_1,k_2} \vec{c}^\dagger(k_1) H^{0}(k_1,k_2) \vec{c}(k_2)$$

# Here, $\vec{c}^\dagger(k)$ and $\vec{c}(k)$ are vectors containing the creation and annihilation operators for all layers and valleys, and $H^{0}(k_1,k_2)$ is the Fourier transform of the noninteracting Hamiltonian:

# $$H^{0}(k_1,k_2) = \frac{1}{V} \int dr e^{-i (k_1-k_2)\cdot r} H^{0}(r)$$

# This final expression represents the transformed noninteracting Hamiltonian in the second quantized form in the momentum space.

# Use the following conventions for the symbols:  
# $l$ : layer index   
# $t$ : top layer  
# $b$ : bottom layer  
# $\tau$ : valley index  
# $+K$ : +K valley  
# $-K$ : -K valley  
# $k$ : momentum operator  
# $E_{l,\tau}$ : energy dispersion for layer $l$ and valley $\tau$ 
# $m_l$ : the effective mass for layer $l$  
# $\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
# $\pm\kappa$ : corners of monolayer moire Brillouin zone
# $\Delta_l$: intralayer potential for layer index $l$  
# $\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
# $H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.
# $V_b$ : bottom layer potential strength  
# $\psi_b$ : bottom layer potential phase  
# $g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
# $r$ : real space position  
# $\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
# $w$ : interlayer tunneling strength
# $\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
# $\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
# $H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
# $\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form 
# $c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
# $c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
# $V$ : area of unit cell in the real space  '''

In [ ]:
prompt=r'''You will be instructed to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   
The degrees of freedom of the system are: valley index (+K and -K valley), layer index (top and bottom layer). 
Express the Kinetic Hamiltonian $H_{Kinetic}(r)$ using $E_{\tau,l}$ which are only on the diagonal terms, and arrange the basis in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top).

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{l,\tau}$ : energy dispersion for layer $l$ and valley $\tau$  '''
# prompt=r'''You will be instructed to perform a particle-hole transformation.  
# Define a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$.  
# You should replace $c_{l,\tau}$ with $b_{l,\tau}(k)^\dagger$, and $c_{l,\tau}^\dagger$ with $b_{l,\tau}(k)$. You should follow the EXAMPLE below to apply the particle-hole transformation.  
# You should recall that $\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$.
# Return the $\hat{H}^{0}$ in the hole operators.

# Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
# $b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$

# ===  
# EXAMPLE:  
# Give a Hamiltonian  $\hat{H}=\sum_{k_1,k_2} c^\dagger(k_1) h(k_1,k_2) c(k_2)$ , and the particle-hole transformation as $b(k)=c^\dagger(k)$. The transformed Hamiltonian is $\hat{H}=\sum_{k_1,k_2} b(k_1) h(k_1,k_2) b^\dagger(k_2)$'''

In [ ]:
response=r'''Given the degrees of freedom and the symbols defined, the kinetic Hamiltonian for a single particle in real space can be expressed as a 4x4 diagonal matrix:

$$H_{Kinetic}(r) = \begin{bmatrix}
E_{b,+K} & 0 & 0 & 0 \\
0 & E_{t,+K} & 0 & 0 \\
0 & 0 & E_{b,-K} & 0 \\
0 & 0 & 0 & E_{t,-K}
\end{bmatrix}$$

This Hamiltonian describes the energy of a single particle in the system, where the diagonal elements represent the energy dispersion for each combination of layer and valley index. For instance, $E_{b,+K}$ corresponds to the energy of a particle in the +K valley of the bottom layer, while $E_{t,-K}$ refers to the energy of a particle in the -K valley of the top layer. The off-diagonal elements are zero, reflecting the fact that these terms would involve transitions between different layers or valleys, which are not part of the kinetic energy term.'''
# response=r'''The real space to momentum space transformation of the total noninteracting Hamiltonian $\hat{H}^{0}$ can be carried out following the example given and considering the additional complexity of having different layers and valleys. 

# The Fourier transforms for the creation and annihilation operators in this multilayer system are given by:

# $$c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$$
# $$c_{\tau,l}(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}(r) e^{-i k \cdot r}$$

# Then, applying the inverse Fourier transforms for the operators in real space gives:

# $$\psi_{\tau,l}^\dagger(r) = \frac{1}{\sqrt{V}} \sum_{k_1} c_{\tau,l}^\dagger(k_1) e^{-i k_1 \cdot r}$$
# $$\psi_{\tau,l}(r) = \frac{1}{\sqrt{V}} \sum_{k_2} c_{\tau,l}(k_2) e^{i k_2 \cdot r}$$

# Therefore, the total noninteracting Hamiltonian in the second quantized form transforms to momentum space as follows:

# $$\hat{H}^{0} = \sum_r \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$$
# $$= \int dr \frac{1}{V} \sum_{k_1, k_2} \vec{c}^\dagger(k_1) e^{-i k_1 \cdot r} H^{0}(r) \vec{c}(k_2) e^{i k_2 \cdot r}$$
# $$= \sum_{k_1,k_2} \vec{c}^\dagger(k_1) \frac{1}{V} \int dr e^{-i (k_1-k_2)\cdot r} H^{0}(r) \vec{c}(k_2)$$
# $$= \sum_{k_1,k_2} \vec{c}^\dagger(k_1) H^{0}(k_1,k_2) \vec{c}(k_2)$$

# Where $\vec{c}^\dagger(k)$ and $\vec{c}(k)$ are vectors containing the creation and annihilation operators for all layers and valleys, and $H^{0}(k_1,k_2)$ is the Fourier transform of the noninteracting Hamiltonian, given by:

# $$H^{0}(k_1,k_2) = \frac{1}{V} \int dr e^{-i (k_1-k_2)\cdot r} H^{0}(r)$$

# This is the transformed total noninteracting Hamiltonian $\hat{H}^{0}$ in the second quantized form in the momentum space.'''

In [19]:
'''Below is a chat history about a physical question showing after ```. 
The format is 1. Background of the question. 2: Question. 3: Answer.
You should summarize the history. 
You should remove redundant information, and keep the main physics,especially, paying attention to the latex equations.
Return the summarized text. 

```
**Background**
{}

**Question**
{}

**Answer**
{}
'''

'Below is a chat history about a physical question showing after ```. \nThe format is 1. Background of the question. 2: Question. 3: Answer.\nYou should summarize the history. \nYou should remove redundant information, and keep the main physics,especially, paying attention to the latex equations.\nReturn the summarized text. \n\n```\n**Background**\n{}\n\n**Question**\n{}\n\n**Answer**\n{}\n\n'

In [ ]:
The question concerns the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2. The system's degrees of freedom include valley index (+K and -K valley), and layer index (top and bottom layer). The kinetic Hamiltonian, $H_{Kinetic}(r)$, expressed using $E_{\tau,l}$, is arranged with the basis order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). The kinetic Hamiltonian, represented as a 4x4 diagonal matrix, is as follows:

$$H_{Kinetic}(r) = \begin{bmatrix}
E_{b,+K} & 0 & 0 & 0 \\
0 & E_{t,+K} & 0 & 0 \\
0 & 0 & E_{b,-K} & 0 \\
0 & 0 & 0 & E_{t,-K}
\end{bmatrix}$$

The diagonal elements depict the energy dispersion for each combination of layer and valley index, while off-diagonal elements being zero indicate no transitions between different layers or valleys in the kinetic energy term.

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{l,\tau}$ : energy dispersion for layer $l$ and valley $\tau$   

In [39]:
print(prompts[7]['content'])

You will be instructed to perform a particle-hole transformation.  
Define a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$.  
You should replace $c_{l,\tau}$ with $b_{l,\tau}(k)^\dagger$, and $c_{l,\tau}^\dagger$ with $b_{l,\tau}(k)$. You should follow the EXAMPLE below to apply the particle-hole transformation.  
You should recall that $\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$.
Return the $\hat{H}^{0}$ in the hole operators.

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$

===  
EXAMPLE:  
Give a Hamiltonian  $\hat{H}=\sum_{k_1,k_2} c^\dagger(k_1) h(k_1,k_2) c(k_2)$ , and the particle-hol

In [25]:
'''
You will be first shown a summary of background of the question.
You will then be asked a further question.
If you find conflicts between the question and background, trust the information in the question
'''

'\nYou will be first shown a summary of background of the question.\nYou will then be asked a further question.\nIf you find conflicts between the question and background, trust the information in the question\n'

In [ ]:
'''
You are a physicist assisting me in constructing a Hamiltonian and/or performing Hartree-Fock calculations, adhering meticulously to my instructions. Your responses should be succinct yet comprehensive, without incorporating any unnecessary information. When needed, you will learn and understand relevant background knowledge through examples.

First, you will be presented with a background summary pertaining to the question. Following this, you will be posed with a question. Your responses should integrate information from both the background and the question.

If you encounter any inconsistencies between the information in the question and the background, prioritize the information provided in the question.
'''

In [ ]:
'''
From the instructions, the energy dispersion for each layer and valley is defined as $E_{\tau,l}(k)=-\frac{\hbar^2 k^2}{2 m_l}$. For the top layer, there is a shift in the momentum $k$ to make the band bottom at $\tilde{\tau}\kappa$. Here, $\tilde{\tau}$ is +1 for $\tau=+K$ and -1 for $\tau=-K$.

With the momentum operator $k$ defined as $k=-i \partial_r$, the expressions for four $E_{\tau,l}(k)$ in the Kinetic Hamiltonian are as follows:

1. For $E_{+K,b}(k)$, the energy dispersion for the bottom layer and +K valley is: $E_{+K,b}(k) = -\frac{\hbar^2 k^2}{2 m_b}$

2. For $E_{+K,t}(k)$, the energy dispersion for the top layer and +K valley is: $E_{+K,t}(k) = -\frac{\hbar^2 (k-\kappa)^2}{2 m_t}$

3. For $E_{-K,b}(k)$, the energy dispersion for the bottom layer and -K valley is: $E_{-K,b}(k) = -\frac{\hbar^2 k^2}{2 m_b}$

4. For $E_{-K,t}(k)$, the energy dispersion for the top layer and -K valley is: $E_{-K,t}(k) = -\frac{\hbar^2 (k+\kappa)^2}{2 m_t}$

Substituting these energy dispersions into the Kinetic Hamiltonian $H_{Kinetic}(r)$, we get:

$$H_{Kinetic}(r) = \begin{bmatrix}
-\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\
0 & -\frac{\hbar^2 (k-\kappa)^2}{2 m_t} & 0 & 0 \\
0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\
0 & 0 & 0 & -\frac{\hbar^2 (k+\kappa)^2}{2 m_t}
\end{bmatrix}$$

Please note that $k$ here represents the momentum operator, which should be replaced with $-i \partial_r$ when calculating in the real space.
'''

In [ ]:
"""
The discussion revolves around the kinetic term of the Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 system with degrees of freedom including the valley index and layer index. The kinetic Hamiltonian, denoted as $H_{Kinetic}(r)$, is a 4x4 diagonal matrix representing the energy dispersion for each layer and valley index.

The energy dispersions, represented as $E_{\tau,l}(k)$, are defined as $E_{\tau,l}(k)=-\frac{\hbar^2 k^2}{2 m_l}$. For the top layer, there is a shift in momentum, so $E_{+K,t}(k) = -\frac{\hbar^2 (k-\kappa)^2}{2 m_t}$ and $E_{-K,t}(k) = -\frac{\hbar^2 (k+\kappa)^2}{2 m_t}$. Here, $\tilde{\tau}$ corresponds to the valley index, taking +1 for $+K$ and -1 for $-K$. The bottom layer doesn't have a momentum shift.

Substituting these energy dispersions into the Kinetic Hamiltonian results in:

$$H_{Kinetic}(r) = \begin{bmatrix}
-\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\
0 & -\frac{\hbar^2 (k-\kappa)^2}{2 m_t} & 0 & 0 \\
0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\
0 & 0 & 0 & -\frac{\hbar^2 (k+\kappa)^2}{2 m_t}
\end{bmatrix}$$

Here, $k$ is a momentum operator, defined as $k=-i \partial_r$, and is used for shorthand notation. It should be replaced with $-i \partial_r$ when calculating in real space.
"""

In [7]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
kwargs_list=[]